## FRAUD DETECTION IN ELECTRICITY AND GAS CONSUMPTION CHALLENGE

* Import Libraries
* Basic EDA
* feature Engineering
* Modelling
* Prediction
* Submission

Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import time

# seed = 47


In [ ]:
invoice_test = pd.read_csv("../test/invoice_test.csv")

invoice_train = pd.read_csv("/home/eben-success/Desktop/Local/Zindi/Fraud Detection in Electricity and Gas Consumption/train/invoice_train.csv")

client_test = pd.read_csv("")
client_train = pd.read_cs